In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# 데이터 불러오기

train = datasets.MNIST(root = "/data", train = True, transform = transforms.ToTensor(), download = True)
test = datasets.MNIST(root = "/data", train = False, transform = transforms.ToTensor(), download = True)

train_dataloader = DataLoader(train, batch_size = 100, shuffle = True)
test_dataloader = DataLoader(test, batch_size = 1000, shuffle = False)


100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:03<00:00, 2.94MB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 157kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:07<00:00, 223kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 8.96MB/s]


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        # 각 커스텀 함수에 쓰일 W, b 값들을 일일히 전부 선언
        # optimizer에서 backward(역전파) 작업을 할 때 torch.nn.Parameter(torch.tensor())를 수정해야 할 값으로 인식해서 수정 작업을 실행
        self.f_weight = nn.Parameter(torch.randn(128, 28*28))
        self.f_bias = nn.Parameter(torch.randn(128))
        self.g_weight = nn.Parameter(torch.randn(64,128))
        self.g_bias = nn.Parameter(torch.randn(64))
        self.h_weight = nn.Parameter(torch.randn(10, 64))
        self.h_bias = nn.Parameter(torch.randn(10))
    def forward(self, x) :
        # 커스텀 함수를 이용해서 계산
        x = torch.matmul(x, self.f_weight.T) + self.f_bias #행렬끼리의 곱셈
        x = nn.functional.relu(x)
        x = nn.functional.dropout(x ,p = 0.25, training = True)
        x = torch.matmul(x, self.g_weight.T) + self.g_bias
        x = nn.functional.relu(x)
        x = torch.matmul(x, self.h_weight.T) + self.h_bias
        return x

#함수 선언
F = NN()
F = F.to(device) #F 전체에 배정된 기기를 .to() 함수를 이용해서 변경이 가능다다
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 0.009)
epoch = 30

F.train() #dropout이 있으니 켜기
for e in range(epoch) :
    loss_sum = 0
    for x, t in train_dataloader :
        x = x.to(device)
        t = t.to(device)
        x = x.reshape(-1,28*28)
        # 신경망 순전파
        y = F(x)
        #y, t비교
        loss = loss_func(y, t)
        loss_sum += loss
        #역전파 (F(x) 수정)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch {e+1} | loss {loss_sum / len(train_dataloader)}")

epoch 1 | loss 31.292280197143555
epoch 2 | loss 3.9358623027801514
epoch 3 | loss 2.1430070400238037
epoch 4 | loss 1.7842055559158325
epoch 5 | loss 1.6035614013671875
epoch 6 | loss 1.5296568870544434
epoch 7 | loss 1.4797770977020264
epoch 8 | loss 1.4275163412094116
epoch 9 | loss 1.4012646675109863
epoch 10 | loss 1.3163130283355713
epoch 11 | loss 1.2952600717544556
epoch 12 | loss 1.2638815641403198
epoch 13 | loss 1.2408316135406494
epoch 14 | loss 1.1969574689865112
epoch 15 | loss 1.1806986331939697
epoch 16 | loss 1.1482630968093872
epoch 17 | loss 1.1348750591278076
epoch 18 | loss 1.1135013103485107
epoch 19 | loss 1.077126145362854
epoch 20 | loss 1.064884066581726
epoch 21 | loss 1.0606322288513184
epoch 22 | loss 1.0207184553146362
epoch 23 | loss 1.0242085456848145
epoch 24 | loss 0.9901885986328125
epoch 25 | loss 0.9859237670898438
epoch 26 | loss 0.9653562903404236
epoch 27 | loss 0.9641261100769043
epoch 28 | loss 0.9449000358581543
epoch 29 | loss 0.9461858272552

In [9]:
F.train()
for e in range(epoch) :
    loss_sum = 0
    for x, t in train_dataloader :
        x = x.to(device)
        t = t.to(device)
        x = x.reshape(-1,28*28)
        y = F(x)

        loss = loss_func(y, t)
        loss_sum += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch {e+1} | loss {loss_sum / len(train_dataloader)}")

epoch 1 | loss 0.9155967235565186
epoch 2 | loss 0.9052784442901611
epoch 3 | loss 0.8765234351158142
epoch 4 | loss 0.8679262399673462
epoch 5 | loss 0.865960419178009
epoch 6 | loss 0.8494653105735779
epoch 7 | loss 0.8514023423194885
epoch 8 | loss 0.8232221603393555
epoch 9 | loss 0.8278650641441345
epoch 10 | loss 0.8208258748054504
epoch 11 | loss 0.8217416405677795
epoch 12 | loss 0.8051092028617859
epoch 13 | loss 0.7994210720062256
epoch 14 | loss 0.7929670214653015
epoch 15 | loss 0.7800220251083374
epoch 16 | loss 0.7737788558006287
epoch 17 | loss 0.764521598815918
epoch 18 | loss 0.754402756690979
epoch 19 | loss 0.7375864386558533
epoch 20 | loss 0.7507950663566589
epoch 21 | loss 0.7467626333236694
epoch 22 | loss 0.7402790188789368
epoch 23 | loss 0.7296691536903381
epoch 24 | loss 0.726026177406311
epoch 25 | loss 0.7208068370819092
epoch 26 | loss 0.7139477133750916
epoch 27 | loss 0.717241108417511
epoch 28 | loss 0.7095756530761719
epoch 29 | loss 0.7051421403884888

In [10]:
cnt = 0
total = 0

F.eval()
for x, t in test_dataloader :
    x = x.to(device)
    t = t.to(device)
    x = x.reshape(-1,28*28)
    y = F(x)
    for i in range(len(x)) :
        if torch.argmax(y[i]) == t[i] :
            cnt += 1
        total += 1

print(cnt / total)

0.7903
